In [1]:
import re
import bs4
import json
import pandas as pd
from tqdm import tqdm
from glob import glob
from bs4 import BeautifulSoup

from blog_fake_review.parser import extract_article_body, refine_paragraph_tag, extract_paragraph, extract_additional_info

In [2]:
origins = glob("./data/origin/*.json")
origins

['./data/origin/홍대 회식 맛집.json',
 './data/origin/홍대 데이트 맛집.json',
 './data/origin/성수 데이트 맛집.json',
 './data/origin/압구정 회식 맛집.json',
 './data/origin/강남역 데이트 맛집.json',
 './data/origin/영등포 회식 맛집.json',
 './data/origin/성수 회식 맛집.json',
 './data/origin/압구정 데이트 맛집.json',
 './data/origin/강남역 회식 맛집.json',
 './data/origin/범계 회식 맛집.json',
 './data/origin/범계 데이트 맛집.json',
 './data/origin/영등포 데이트 맛집.json']

In [3]:
parsed_list = glob("./data/semi-structured/*.json")
parsed_list = [p.split('/')[-1] for p in parsed_list]
parsed_list

[]

In [4]:
comment_code = re.compile(r'<!--.*?-->')

In [5]:
for path in origins:
    file_name = path.split('/')[-1]
    
    if file_name in parsed_list: continue
        
    with open(path, "r", encoding='utf-8') as f:
        blog = json.load(f)

    result = []
    for row in tqdm(blog, desc=file_name.split('.')[0]):
        data = {
            "blog_name": row["blog_name"],
            "date": row["date"],
            "url": row["url"],
            "title": row["title"]
        }
        
        soup = BeautifulSoup(comment_code.sub('', row['origin_source']), 'html.parser')
        version, body_tag = extract_article_body(soup)
        
        if not body_tag: continue
            
        tags = extract_paragraph(body_tag, version)
        
        data["contents"] = sum([refine_paragraph_tag(tag) for tag in tags], [])
        
        data["additional"] = extract_additional_info(soup)
        result.append(data)
    
    with open(f'./data/semi-structured/{file_name}', 'w', encoding='utf-8') as f:
        json.dump(result, f)

영등포 데이트 맛집: 100%|██████████| 979/979 [01:14<00:00, 13.13it/s]
